In [1]:
import pandas as pd
import numpy as np
import json
import unicodedata

In [2]:
doc = pd.read_csv('data/ica_data.csv', encoding='ISO-8859-1', error_bad_lines=False, delimiter=';')
doc.head()

,ID,Timestamp,Tweet,Keluhan,Respon,Bukan Keluhan/Respon,Topik Umum,Topik Spesifik,Lokasi,Waktu,Unnamed: 10
0,"6,78845E+17","21 Des 2015, 02.50.25 PM",@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
1,"6,78846E+17","21 Des 2015, 02.53.02 PM",@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN,Lingkungan Hidup,pohon tumbang,Jalan Sangkuriang depan Polsek Coblong,-,NaN
2,"6,78847E+17","21 Des 2015, 02.57.55 PM",Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
3,"6,78847E+17","21 Des 2015, 02.58.54 PM",@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN,Infrastruktur,lampu penerangan jalan umum,Jalan depan Kampus LPKIA,-,NaN
4,"6,78849E+17","21 Des 2015, 03.06.49 PM","@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN


In [3]:
data = doc.loc[:, 'Tweet':'Bukan Keluhan/Respon']
data.head()

,Tweet,Keluhan,Respon,Bukan Keluhan/Respon
0,@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya
1,@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN
2,Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya
3,@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN
4,"@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya


In [4]:
# data = data.fillna('0')
# data['Keluhan'] = data['Keluhan'].map({'Ya':1})
# data['Respon'] = data['Respon'].map({'Ya':1})
# data['Bukan Keluhan/Respon'] = data['Bukan Keluhan/Respon'].map({'Ya':1})

In [5]:
# data_text = data.loc[:, 'Tweet']
data_tweet = data.loc[:, 'Tweet']

In [6]:
# data_keluhan = data.loc[:, 'Keluhan']
# data_keluhan[0:3]

In [23]:
# get index with "Ya" value in 3 column
temp = data['Keluhan'].values.tolist()
idx_keluhan = [i for i, x in enumerate(temp) if x == 'Ya']

temp = data['Respon'].values.tolist()
idx_respon = [i for i, x in enumerate(temp) if x == 'Ya']

temp = data['Bukan Keluhan/Respon'].values.tolist()
idx_notboth = [i for i, x in enumerate(temp) if x == 'Ya']

# create 1 column for label
# downsampling, get as much as the smallest data
data_label = []
data_temp = []

minim = min(len(idx_keluhan), len(idx_respon), len(idx_notboth))

for i in range (0, minim):
    data_temp.append(data_tweet[idx_keluhan[i]])
    data_label.append(0)
for i in range (0, minim):
    data_temp.append(data_tweet[idx_respon[i]])
    data_label.append(1)
for i in range (0, minim):
    data_temp.append(data_tweet[idx_notboth[i]])
    data_label.append(2)
    
data_tweet = data_temp
len(data_label)

2751

In [8]:
import requests
import re
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [9]:
def removeURL(sent):
    # remove username
    sent = re.sub('RT @[^\s]+','',sent)
    sent = re.sub('@[^\s]+','',sent)
    sent = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', sent, flags=re.MULTILINE)
    sent = re.sub('http[s]?', '', sent, flags=re.MULTILINE)
    return sent

In [10]:
def normalization(sent):
    # text normalization
    header = {
        'Content-Type' : 'application/json',
        'x-api-key' : 'PGSpcyOiDPqdlNmQ1kd66p59qhTDuLUOCqdJk4sF'
    }
    req = requests.post("https://api.prosa.ai/v1/normals", headers=header, json = { 'text' : sent})
    res = req.json()
    print(res)
    return res

In [11]:
def stemmer(sent):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(sent)

In [12]:
def tokenizer(sent):
    words = word_tokenize(sent)
#     print (words)
    return words

In [13]:
def removeStopWord(sent):
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    return stopword.remove(sent)

In [15]:
n_tweet_per_call = 200 # jumlah tweet untuk setiap call normalisasi
# n_call = (len(data)/940)
n_call = 1
data_text = []

for i in range (0, n_call):
#     k = i*n_tweet_per_call
    str_temp = ''
          
    for j in range (i*n_tweet_per_call, (i+1)*n_tweet_per_call):
        str_temp = str_temp + " | " + data_tweet[j]
    
    str_temp = removeURL(str_temp)
    text = normalization(str_temp)['text']
    text = text.encode('ascii', 'ignore')
    temp = text.split('|')
    
    for k in range (1, len(temp)):
        data_text.append(stemmer(temp[k]))
            
print data_text
          

{u'text': u'|ya mungkin karena perempuan bj merah yang paling kelihatan karena berada di depan, laki-laki juga sering jadi bahan joke RK|kang teman saya tertimpa pohn dijln sangkuriang depan polsek coblong tolong ditertibkan pohon yang sudah lapuknuhun|Di tribun jabar biasanya suka di pos agenda kang emil. tapi tidak setiap hari sih. kalau saya kebandung bisa bertemu Kang di mana|Lapor pak lampu penerangan jalan depan kampus lpkia ngaplek :( :( :(|bapak/ibu mau tanya, kalau pemeliharaan taman-taman yang banyak dibagun sekarang bagaimana nantinya? |Pa tempat kerja saya tanggal 25 tetap masuk. bagaimana itu aturannya pa? kalau misalnya enggak masuk dipotong gaji|00106700466, punten sudah seminggu air PDAM tidak mengalir. Mohon bantuannya segera. |Anak SMA yang ketahuan mencuri helem di bip, hari ini pak|ternyata sudah ada warga yang konfirm ke pdam, katanya: air tidak akan mengalir selama 1 bulan karena ada perbaikan|Kang jalan di Muararajeun Baru, baru sebulanan dibenerin, sudah rusak l

['ya mungkin karena perempuan bj merah yang paling lihat karena ada di depan laki juga sering jadi bahan joke rk', 'kang teman saya timpa pohn dijln sangkuriang depan polsek coblong tolong tertib pohon yang sudah lapuknuhun', 'di tribun jabar biasa suka di pos agenda kang emil tapi tidak tiap hari sih kalau saya bandung bisa temu kang di mana', 'lapor pak lampu terang jalan depan kampus lpkia ngaplek', 'bapak ibu mau tanya kalau pelihara taman yang banyak dibagun sekarang bagaimana nanti', 'pa tempat kerja saya tanggal 25 tetap masuk bagaimana itu atur pa kalau misal enggak masuk potong gaji', '00106700466 punten sudah minggu air pdam tidak alir mohon bantu segera', 'anak sma yang tahu curi hem di bip hari ini pak', 'nyata sudah ada warga yang konfirm ke pdam kata air tidak akan alir lama 1 bulan karena ada baik', 'kang jalan di muararajeun baru baru bulan dibenerin sudah rusak lagi yang kerja seperti enggak serius', 'didinyamah kidu bosss ku maha pak daerah sapharuhai yeuh kang parkir

In [16]:
for i in range (0, len(data_text)):
    data_text[i] = removeStopWord(data_text[i])
data_text

['mungkin perempuan bj merah paling lihat ada depan laki sering jadi bahan joke rk',
 'kang teman timpa pohn dijln sangkuriang depan polsek coblong tertib pohon sudah lapuknuhun',
 'tribun jabar biasa suka pos agenda kang emil tidak tiap hari sih kalau bandung temu kang mana',
 'lapor pak lampu terang jalan depan kampus lpkia ngaplek',
 'bapak ibu mau tanya kalau pelihara taman banyak dibagun sekarang bagaimana',
 'pa tempat kerja tanggal 25 tetap masuk bagaimana atur pa kalau misal enggak masuk potong gaji',
 '00106700466 punten minggu air pdam alir mohon bantu segera',
 'anak sma tahu curi hem bip hari pak',
 'nyata ada warga konfirm pdam kata air akan alir lama 1 bulan ada baik',
 'kang jalan muararajeun baru baru bulan dibenerin rusak yang kerja enggak serius',
 'didinyamah kidu bosss ku maha pak daerah sapharuhai yeuh kang parkir liar seeur pis atuhda- -',
 'pak punten marahin motor enggak pakai spakbor belakang mah td hujan wajah rupawan kacepretan',
 'hatur nuhun dbmp lampu jala

In [25]:
# temp = text.split('|')
# temp

In [24]:
# without normalisation
data_temp = []
for tweet in data_tweet:
    tweet = stemmer(removeStopWord(removeURL(tweet.encode('utf-8'))))
    data_temp.append(tweet)
data_tweet = data_temp

In [ ]:
# data_temp = []
# for tweet in data_tweet:
#     tweet = tokenizer(tweet)
#     data_temp.append(tweet)
# data_tweet = data_temp

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import neighbors, svm, tree
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

In [ ]:
model = Word2Vec(data_tweet, min_count=1)
model.train(data_tweet, total_examples=1, epochs=1)
x = model.wv

In [28]:
text_train, text_test, y_train, y_test = train_test_split(data_tweet, data_label, test_size=0.25, random_state=42)

In [30]:
tfidf = TfidfVectorizer()

X_tfidf_train = tfidf.fit_transform(text_train)
X_tfidf_test = tfidf.fit_transform(text_test)

In [36]:
# svc = svm.LinearSVC()
# svc.fit(X_tfidf_train, y_train)
# svc_pred = svc.predict(X_tfidf_test)
# accuracy_score(svc_pred, y_test)